In [33]:
import pycaret
import pandas as pd
import os
import sys 
import shutil
import pickle

In [10]:
sys.path.append(os.path.abspath("/Users/raj2.gaurav/Desktop/supervised_classification/09. Config"))
import config

In [11]:
# import pycaret classification and init setup
from pycaret.classification import *

In [12]:
path=config.train_data_path

In [13]:
data=pd.read_pickle(path)

In [14]:
data.shape

(100000, 28)

In [15]:
data.columns

Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')

In [16]:
Target = config.Target
Train_size = config.Train_size
Ignore_features = config.Ignore_features
Preprocess = config.Preprocess
Imputation_type = config.Imputation_type
numeric_imputation = config.numeric_imputation
categorical_imputation = config.categorical_imputation
remove_outliers = config.remove_outliers
outliers_method = config.outliers_method
outliers_threshold = config.outliers_threshold
fix_imbalance = config.fix_imbalance
fix_imbalance_method = config.fix_imbalance_method
normalize = config.normalize
normalize_method = config.normalize_method

In [17]:
s = setup(data, target = Target, train_size = Train_size, ignore_features = Ignore_features,
          preprocess = Preprocess,imputation_type = Imputation_type, numeric_imputation = numeric_imputation,
         categorical_imputation = categorical_imputation, remove_outliers = remove_outliers, 
          outliers_method = outliers_method, outliers_threshold = outliers_threshold,
         fix_imbalance = fix_imbalance, fix_imbalance_method = fix_imbalance_method,
         normalize = normalize, normalize_method = normalize_method)

,Description,Value
0,Session id,8216
1,Target,Credit_Score
2,Target type,Binary
3,Original data shape,"(100000, 28)"
4,Transformed data shape,"(87558, 49)"
5,Transformed train set shape,"(57558, 49)"
6,Transformed test set shape,"(30000, 49)"
7,Ignore features,5
8,Numeric features,13
9,Categorical features,9


In [39]:
get_config('X_train')

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,...,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
12135,49,Entrepreneur,123299.117188,4194.170898,2,2,8,100,"Not Specified, Credit-Builder Loan, and Credit...",10,...,7.0,Good,845.469971,32.322124,NaN,No,289.4723048639328,NaN,High_spent_Large_value_payments,851.332825
46110,37,Engineer,57227.039062,4561.919922,9,9,16,7,"Not Specified, Credit-Builder Loan, Auto Loan,...",39,...,9.0,Bad,3902.969971,37.820419,101.0,Yes,220.27681374800392,182.419662,Low_spent_Large_value_payments,323.495514
68528,35,Engineer,9743.959961,889.996643,8,7,4,4,"Payday Loan, Debt Consolidation Loan, Not Spec...",20,...,0.0,_,625.729980,28.733507,196.0,No,18.85845156233991,66.285095,Low_spent_Small_value_payments,293.856110
48736,17,Journalist,8205.139648,962.761658,9,9,34,8,"Auto Loan, Auto Loan, Payday Loan, Home Equity...",36,...,6.0,Bad,3176.860107,36.307137,2.0,Yes,53.99910271431013,86.661674,Low_spent_Medium_value_payments,235.615387
694,22,_______,66105.398438,5653.783203,6,8,31,5,"Mortgage Loan, Auto Loan, Not Specified, Stude...",26,...,12.0,Bad,1549.560059,38.648224,140.0,NM,214.07212305606225,274.572662,Low_spent_Small_value_payments,366.733551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97149,34,Manager,24873.099609,1848.758301,3,6,8,3,"Auto Loan, Mortgage Loan, and Not Specified",23,...,6.0,_,30.309999,29.124367,67.0,Yes,39.84069977161735,56.810616,Low_spent_Medium_value_payments,368.224518
89704,38,Entrepreneur,76519.296875,6277.608398,5,7,10,4,"Student Loan, Home Equity Loan, Auto Loan, and...",5,...,3.0,Good,260.850006,36.584496,380.0,No,228.5434193935592,189.472275,High_spent_Medium_value_payments,459.745148
7724,50,Doctor,24667.160156,2284.596680,8,6,16,3,"Debt Consolidation Loan, Mortgage Loan, and Ho...",8,...,5.0,Standard,722.020020,27.607992,235.0,Yes,49.23739807081449,67.565186,High_spent_Medium_value_payments,361.657074
45557,21,Developer,17137.570312,1617.130859,8,7,34,9,"Debt Consolidation Loan, Student Loan, Debt Co...",22,...,16.0,_,3680.600098,30.934248,110.0,Yes,119.5320319672674,122.757576,Low_spent_Small_value_payments,209.423477


In [18]:
report=pull()

In [19]:
Data_Sense=report.iloc[1:10]

In [20]:
models = config.models

In [21]:
best_model = compare_models(include = models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8803,0.9273,0.8319,0.7729,0.8012,0.7158,0.7168,1.5330
lr,Logistic Regression,0.8622,0.9246,0.9003,0.7061,0.7913,0.6907,0.7023,1.5550
ada,Ada Boost Classifier,0.8511,0.9150,0.8948,0.6925,0.7788,0.6701,0.6846,1.2560
lightgbm,Light Gradient Boosting Machine,0.8144,0.8658,0.7070,0.6730,0.6880,0.5563,0.5580,0.4170
dt,Decision Tree Classifier,0.7806,0.7327,0.6186,0.6358,0.6225,0.4689,0.4724,0.4120


Processing:   0%|          | 0/25 [00:00<?, ?it/s]

In [22]:
Compare_Model_Report=pull()

In [23]:
if not os.path.exists(config.data_hygeine_path):
    os.makedirs(config.data_hygeine_path)
writer2 = pd.ExcelWriter(config.data_hygeibe_report_file)
Data_Sense.to_excel(writer2, sheet_name = config.data_hygeibe_report_file_sheet1, index = False)
Compare_Model_Report.to_excel(writer2, sheet_name = config.data_hygeibe_report_file_sheet2, index = False)
writer2.close()

In [24]:
best_5=Compare_Model_Report.head()

In [29]:
best_5

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8803,0.9273,0.8319,0.7729,0.8012,0.7158,0.7168,1.533
lr,Logistic Regression,0.8622,0.9246,0.9003,0.7061,0.7913,0.6907,0.7023,1.555
ada,Ada Boost Classifier,0.8511,0.9150,0.8948,0.6925,0.7788,0.6701,0.6846,1.256
lightgbm,Light Gradient Boosting Machine,0.8144,0.8658,0.7070,0.6730,0.6880,0.5563,0.5580,0.417
dt,Decision Tree Classifier,0.7806,0.7327,0.6186,0.6358,0.6225,0.4689,0.4724,0.412


In [40]:
for i in config.models:
    print(i)
    model1=create_model(i,fold=config.fold_n)
    Model1_Report=pull()
    tuned_model = tune_model(model1,fold=config.fold_tuned_n)
    Model1_Tuned_Report=pull()
    Hyper_para=str(tuned_model)
    Model1_Hyperparamters=pd.DataFrame(Hyper_para.split('(')[1].split())
    model_name=i
    if not os.path.exists(eval(config.model_hygeine_path)):
        os.makedirs(eval(config.model_hygeine_path))
        writer2 = pd.ExcelWriter(eval(config.model_hygeine_report_file))
        Model1_Report.to_excel(writer2, sheet_name = config.model_hygeibe_report_file_sheet1, index = False)
        Model1_Tuned_Report.to_excel(writer2, sheet_name = config.model_hygeibe_report_file_sheet2, index = False)
        Model1_Hyperparamters.to_excel(writer2, sheet_name = config.model_hygeibe_report_file_sheet3, index = False)
        writer2.close()
    lst_plot=config.plot_lst
    for j in lst_plot:
        plot_model(tuned_model, plot = j,save=True)
    if not os.path.exists(eval(config.plot_path)):
        os.makedirs(eval(config.plot_path))

    source = os.path.join(config.plot_soruce_path)
    sort = os.path.join(eval(config.plot_path))

    for f in os.listdir(source):
        if f.endswith(config.plot_format):
            shutil.move(os.path.join(source, f), sort)
    if not os.path.exists(eval(config.save_model_path)):
        os.makedirs(eval(config.save_model_path))
    save_model(tuned_model, f'{eval(config.save_model_path)}/{i}', verbose=True)
    with open( f'{eval(config.save_model_path)}/{i}.pickle', 'wb') as files:
        pickle.dump(tuned_model, files)

lr


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8584,0.9249,0.8965,0.6997,0.7860,0.6826,0.6943
1,0.8541,0.9214,0.8958,0.6920,0.7808,0.6742,0.6868
2,0.8359,0.9130,0.9096,0.6567,0.7627,0.6422,0.6621
3,0.8411,0.9111,0.9047,0.6665,0.7675,0.6510,0.6685
4,0.8532,0.9246,0.9128,0.6854,0.7829,0.6754,0.6912
Mean,0.8485,0.9190,0.9039,0.6800,0.7760,0.6651,0.6806
Std,0.0086,0.0058,0.0068,0.0160,0.0092,0.0156,0.0129


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8614,0.9261,0.8985,0.7047,0.7899,0.6888,0.7001
1,0.8593,0.9223,0.8961,0.7015,0.7869,0.6842,0.6956
2,0.8444,0.9140,0.9064,0.6717,0.7716,0.6575,0.6744
3,0.8456,0.9112,0.9010,0.6752,0.7719,0.6588,0.6744
4,0.8576,0.9251,0.9111,0.6939,0.7878,0.6836,0.6980
Mean,0.8537,0.9197,0.9026,0.6894,0.7816,0.6746,0.6885
Std,0.0072,0.0061,0.0055,0.0136,0.0081,0.0136,0.0116


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8827,0.9118,0.8339,0.7776,0.8048,0.7211,0.7220
1,0.8806,0.9086,0.8387,0.7700,0.8029,0.7174,0.7188
2,0.8776,0.9076,0.8443,0.7601,0.8000,0.7121,0.7142
3,0.8802,0.9077,0.8291,0.7740,0.8006,0.7151,0.7160
4,0.8841,0.9123,0.8544,0.7707,0.8104,0.7272,0.7293
Mean,0.8810,0.9096,0.8401,0.7705,0.8037,0.7186,0.7200
Std,0.0022,0.0020,0.0088,0.0059,0.0038,0.0052,0.0053


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


Transformation Pipeline and Model Successfully Saved


In [37]:
tuned_model.predict

<bound method LGBMClassifier.predict of LGBMClassifier(bagging_fraction=1.0, bagging_freq=5, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.6,
               importance_type='split', learning_rate=0.01, max_depth=-1,
               min_child_samples=21, min_child_weight=0.001, min_split_gain=0.3,
               n_estimators=190, n_jobs=-1, num_leaves=256, objective=None,
               random_state=8216, reg_alpha=0.0005, reg_lambda=0.01,
               silent='warn', subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)>

In [25]:
model1=create_model(best_5.index[0],fold=config.fold_n)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8801,0.9254,0.8305,0.7730,0.8007,0.7152,0.7161
1,0.8815,0.9249,0.8458,0.7687,0.8054,0.7205,0.7222
2,0.8740,0.9205,0.8323,0.7573,0.7930,0.7027,0.7044
3,0.8779,0.9251,0.8313,0.7670,0.7979,0.7106,0.7117
4,0.8817,0.9279,0.8379,0.7732,0.8043,0.7197,0.7209
Mean,0.8790,0.9248,0.8356,0.7678,0.8003,0.7137,0.7151
Std,0.0029,0.0024,0.0058,0.0058,0.0045,0.0066,0.0065


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [26]:
Model1_Report=pull()

In [27]:
tuned_model = tune_model(model1,fold=config.fold_tuned_n)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8700,0.9233,0.8500,0.7402,0.7913,0.6975,0.7011
1,0.8715,0.9196,0.8889,0.7281,0.8005,0.7071,0.7148
2,0.8681,0.9184,0.8660,0.7298,0.7921,0.6966,0.7020
3,0.8656,0.9192,0.8682,0.7235,0.7893,0.6918,0.6980
4,0.8747,0.9237,0.8776,0.7392,0.8025,0.7117,0.7173
Mean,0.8700,0.9208,0.8701,0.7321,0.7951,0.7009,0.7066
Std,0.0031,0.0022,0.0129,0.0065,0.0053,0.0073,0.0078


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [28]:
Model1_Tuned_Report=pull()

In [39]:
Hyper_para=str(tuned_model)
Model1_Hyperparamters=pd.DataFrame(Hyper_para.split('(')[1].split())

In [29]:
model_name = best_5['Model'][0]

In [40]:
if not os.path.exists(eval(config.model_hygeine_path)):
    os.makedirs(eval(config.model_hygeine_path))
writer2 = pd.ExcelWriter(eval(config.model_hygeine_report_file))
Model1_Report.to_excel(writer2, sheet_name = config.model_hygeibe_report_file_sheet1, index = False)
Model1_Tuned_Report.to_excel(writer2, sheet_name = config.model_hygeibe_report_file_sheet2, index = False)
Model1_Hyperparamters.to_excel(writer2, sheet_name = config.model_hygeibe_report_file_sheet3, index = False)
writer2.close()

In [ ]:
### Accuracy results

In [44]:
lst_plot=config.plot_lst
for i in lst_plot:
    plot_model(tuned_model, plot = i,save=True) 

In [46]:
if not os.path.exists(eval(config.plot_path)):
    os.makedirs(eval(config.plot_path))


source = os.path.join(config.plot_soruce_path)
sort = os.path.join(eval(config.plot_path))

for f in os.listdir(source):
    if f.endswith(config.plot_format):
        shutil.move(os.path.join(source, f), sort)

In [47]:
finalize_model(tuned_model)

Pipeline(memory=FastMemory(location=/var/folders/3p/7q9_hy6x4q58xrz02_n9tvv0fryx5_/T/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['hour_of_day',
                                             'avg_last_rrc_measurement_earfcn',
                                             'num_distinct_imsi',
                                             'mute_call_gap_indicator',
                                             'num_mute_calls', 'avg_cqi',
                                             'min_cqi', 'max_cqi', 'p05_cqi',
                                             'p10_cqi', 'p50_cqi', 'p90_cqi',
                                             'p95_cq...
                                boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, feature_fraction=0.7,
                                importance_type='split', learning_rate=0.2,
                             

In [48]:
if not os.path.exists(eval(config.save_model_path)):
    os.makedirs(eval(config.save_model_path))

In [50]:
save_model(tuned_model, f'{eval(config.save_model_path)}/{best_5.index[0]}', verbose=True)

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/var/folders/3p/7q9_hy6x4q58xrz02_n9tvv0fryx5_/T/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['hour_of_day',
                                              'avg_last_rrc_measurement_earfcn',
                                              'num_distinct_imsi',
                                              'mute_call_gap_indicator',
                                              'num_mute_calls', 'avg_cqi',
                                              'min_cqi', 'max_cqi', 'p05_cqi',
                                              'p10_cqi', 'p50_cqi', 'p90_cqi',
                                              'p95_cq...
                                 boosting_type='gbdt', class_weight=None,
                                 colsample_bytree=1.0, feature_fraction=0.7,
                                 importance_type='split', learning_rate=0.2,
               

In [51]:
tuned_model

LGBMClassifier(bagging_fraction=0.9, bagging_freq=0, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
               importance_type='split', learning_rate=0.2, max_depth=-1,
               min_child_samples=61, min_child_weight=0.001, min_split_gain=0.1,
               n_estimators=150, n_jobs=-1, num_leaves=40, objective=None,
               random_state=2593, reg_alpha=0.1, reg_lambda=3, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [52]:
 f'{eval(config.save_model_path)}/{best_5.index[0]}'

'/Users/raj2.gaurav/Desktop/Git/Causal_Model/07. Model/Light Gradient Boosting Machine/Model_Results/Model_Objects/lightgbm'

In [57]:
# loading library
import pickle
# create an iterator object with write permission - model.pkl
with open( f'{eval(config.save_model_path)}/{best_5.index[0]}.pickle', 'wb') as files:
    pickle.dump(tuned_model, files)

In [56]:
from joblib import Parallel, delayed

import joblib

# Save the model as a pickle in a file

joblib.dump(tuned_model,  f'{eval(config.save_model_path)}/{best_5.index[0]}.pkl')

['/Users/raj2.gaurav/Desktop/Git/Causal_Model/07. Model/Light Gradient Boosting Machine/Model_Results/Model_Objects/lightgbm.pkl']